In [12]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sbn
import pandas as pd
from datetime import datetime
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from pandas.api.types import CategoricalDtype

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
plt.style.use('default') 
sbn.set(style="whitegrid") 

# **Satinización de archivos de edad y genero**

In [2]:
#Antes que nada, me intriga si los archivos de 'datos_navent' y los de 'hasta15/4' tienen overlap, son redundantes o consecutivos
genero_edad_postulantes = pd.read_csv(
    'datos/datos_navent/fiuba_2_postulantes_genero_y_edad.csv'
).merge(
    pd.read_csv('datos/hasta_15_4/fiuba_2_postulantes_genero_y_edad.csv'),
    on=['idpostulante','fechanacimiento','sexo'],
    how='outer'
)
def postulantes_str2timestamp(s): # debo
    if type(s) is datetime:
        return s
    try:
        return datetime.strptime(s, '%Y-%m-%d')
    except:
        return None
#veo que hay repetidos por id
#veo que hay gente (4) con sexo '0'
# Genero una funcion para satinizar archivos de postulantes_genero_edad
genero_categoria = CategoricalDtype(categories=["MASC","FEM","NO_DECLARA"], ordered=False)
def satinize_postulante_genero_edad(df):
    df['sexo'] = df['sexo'].astype(genero_categoria).fillna("NO_DECLARA")
    df = df.drop_duplicates(subset='idpostulante')
    df['fechanacimiento']= df['fechanacimiento'].map(postulantes_str2timestamp)
    mean_datetime = datetime.now()
    df['fechanacimiento']= df['fechanacimiento'].map(lambda it: it if it else mean_datetime)
    df['edad'] = df.fechanacimiento.apply(lambda it: 2018 - it.year)
    df = df[df.edad >17][df.edad<105]
    return df
genero_edad_postulantes = satinize_postulante_genero_edad(genero_edad_postulantes)
genero_edad_postulantes.shape[0]
educacion_postulantes = pd.read_csv(
    'datos/datos_navent/fiuba_1_postulantes_educacion.csv'
).merge(pd.read_csv('datos/hasta_15_4/fiuba_1_postulantes_educacion.csv'))
print(educacion_postulantes.isna().any()) #no hay nulos
print(educacion_postulantes.duplicated().value_counts())#ni duplicados
def generar_nivel_educativo(postulantes, educacion):
    #Considero como nivel educativo el maximo nivel que alcanzaron que fue completado.
    #Descarto estudios en curso o abandonados, quizas deba volver aca en algun momento
    tipo_estudio = CategoricalDtype(
        categories=["Otro","Secundario","Terciario/Técnico","Universitario","Posgrado","Master","Doctorado"],
        ordered=True
    )
    educacion = educacion[educacion.estado=='Graduado'].drop(['estado'],1)
    educacion.nombre= educacion.nombre.astype(tipo_estudio)
    educacion = educacion.sort_values(by='nombre',ascending=True)
    educacion = educacion.drop_duplicates(subset='idpostulante',keep='last')
    postulantes = postulantes.merge(educacion, on='idpostulante')
    postulantes = postulantes.rename(index=str,columns={'nombre':'nivel_educativo'})
    return postulantes.copy()
postulantes = generar_nivel_educativo(genero_edad_postulantes, educacion_postulantes)
del genero_edad_postulantes

/home/vasectomio/.virtualenv/datos/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/vasectomio/.virtualenv/datos/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/vasectomio/.virtualenv/datos/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

idpostulante    False
nombre          False
estado          False
dtype: bool
False    137072
dtype: int64


In [3]:
vistas = pd.concat([
    pd.read_csv('datos/hasta_15_4/fiuba_3_vistas.csv'),
    pd.read_csv('datos/datos_navent/fiuba_3_vistas.csv')
]).drop_duplicates().rename(columns={'idAviso': 'idaviso'})
vistas.idaviso = vistas.idaviso.astype('int64')
print("nulos: ",vistas.isna().any()) #no hay nulos
print("duplicados: ",vistas.duplicated().value_counts())#ni duplicados

nulos:  idaviso         False
timestamp       False
idpostulante    False
dtype: bool
duplicados:  False    6682624
dtype: int64


In [4]:
def agregar_cantidad_anuncios(df, nombre_columna, anuncios):
    values = pd.DataFrame()
    value_counts = anuncios['idpostulante'].value_counts()
    values['idpostulante'] = value_counts.index
    values[nombre_columna] = value_counts.values
    df = pd.merge(df,values, on='idpostulante',how='left')
    df[nombre_columna] = df[nombre_columna].fillna(0)
    return df
postulantes = agregar_cantidad_anuncios(postulantes, 'anuncios_vistos', vistas)

## postulaciones

In [5]:
postulaciones = pd.read_csv(
    'datos/datos_navent/fiuba_4_postulaciones.csv'
).merge(pd.read_csv('datos/hasta_15_4/fiuba_4_postulaciones.csv'))
print(postulaciones.count)
print(postulaciones.drop_duplicates().count) #no hay duplicados!
print(postulaciones.head())
def sanitize_postulaciones(df):
    postulaciones['date'] = pd.to_datetime(postulaciones['fechapostulacion']).dt.date
    postulaciones['time'] = pd.to_datetime(postulaciones['fechapostulacion']).dt.time
    return postulaciones.copy()
postulaciones = sanitize_postulaciones(postulaciones)
print(postulaciones.head())

<bound method DataFrame.count of             idaviso idpostulante     fechapostulacion
0        1112286523         ZaO5  2018-01-24 15:07:39
1        1112272060         ZaO5  2018-01-24 15:20:10
2        1112288401         ZaO5  2018-01-26 08:37:04
3        1112300563         ZaO5  2018-01-30 13:35:48
4        1112293018         ZaO5  2018-01-31 08:41:05
5        1112301287         ZaO5  2018-01-31 08:55:20
6        1112309589         ZaO5  2018-02-03 15:55:15
7        1112317020         ZaO5  2018-02-07 09:56:22
8        1112322363         ZaO5  2018-02-09 10:29:54
9        1112327122         ZaO5  2018-02-11 17:49:26
10       1112315062         ZaO5  2018-02-11 18:08:14
11       1112348722         ZaO5  2018-02-21 19:53:24
12       1112347076         ZaO5  2018-02-21 20:10:19
13       1112346704         ZaO5  2018-02-21 20:15:35
14       1112346547         ZaO5  2018-02-21 20:17:29
15       1112335122         ZaO5  2018-02-21 20:23:19
16       1112350663         ZaO5  2018-02-21 20:2

## Archivo 'avisos_detalle'

In [6]:
avisos_detalle = pd.concat([
    pd.read_csv('datos/datos_navent/fiuba_6_avisos_detalle.csv'),
    pd.read_csv('datos/hasta_15_4/fiuba_6_avisos_detalle.csv'),
    pd.read_csv('datos/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')
])
#ESTE DF TIENE MUCHOS DATOS NULOS EN LAS COLUMNAS DE 'CIUDAD' y 'MAPACALLE'. 
#Decidimos eliminarlas ya que no nos parecieron muy relevantes para el analisis
#idpais solo tiene valor 1, la descripcion nunca es nula
#avisos_detalle.drop('descripcion',1,inplace=True)
tipo_trabajo = CategoricalDtype(
    categories=["Full-time","Part-time","Teletrabajo","Por Horas","Pasantia","Temporario","Por Contrato","Fines de Semana","Primer empleo"],
    ordered=True #de 'mas fijo' a 'menos fijo'
)
avisos_detalle.nombre_area.value_counts()# hay muchas areas, no van a tener su propia categoria
def sanitize_aviso_detalle(df):
    df['tipo_de_trabajo'] = df['tipo_de_trabajo'].astype(tipo_trabajo)
    #df.titulo = df.titulo.apply(lambda it: it.lower())
    #df.descripcion = df.descripcion.apply(lambda it: it.lower())
    for columna in ['nombre_zona','nombre_area','denominacion_empresa','nivel_laboral']:
        df.loc[:,columna] = df.loc[:,columna].astype('category')
    df.nivel_laboral = df.nivel_laboral.fillna('Otro')
    df = df.drop_duplicates(subset='idaviso')
    return df.drop(['ciudad','idpais','mapacalle'],1)
avisos_detalle = sanitize_aviso_detalle(avisos_detalle)

# Pruebo un random forest, con solamente las variables numericas/categoricas

In [7]:
#no hace falta modificar el anuncio de postulantes
avisos_forest =  avisos_detalle.drop(['titulo','descripcion'],1)
postulaciones_forest = postulaciones.loc[:,['idaviso','idpostulante']]
postulaciones_forest['sepostulo'] = 1
joined = postulantes.drop(['fechanacimiento'],1).merge(
    vistas.loc[:,['idpostulante','idaviso']],on='idpostulante',how='left'
).merge(
    avisos_forest,on='idaviso',how='left'
).merge(
    postulaciones_forest,on=['idaviso','idpostulante'],how='left'
)
joined.sepostulo = joined.sepostulo.fillna(0)
joined['es_entrenamiento'] = np.random.uniform(0, 1, joined.shape[0]) <= 0.75
features = [it for it in joined.columns if it not in ['idaviso','idpostulante','es_entrenamiento','sepostulo']]
categorical_coulumns = joined.select_dtypes(['category']).columns
joined[categorical_coulumns] = joined[categorical_coulumns].apply(lambda it: it.cat.codes)

joined.shape
#grouped = joined.groupby('idpostulante')

(2591014, 13)

In [8]:
features
train, test = joined[joined.es_entrenamiento],joined[~joined.es_entrenamiento]

In [9]:
classifier = RandomForestClassifier(n_jobs=3, random_state=0)
classifier.fit(train[features], train.sepostulo)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=3,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [13]:
results = pd.DataFrame()
results['prediccion'] = classifier.predict(test[features])
results['baseline'] = [0 for i in range(test.shape[0])]
results['verdad'] = test['sepostulo'].astype('int64').values
test.sepostulo.isna().value_counts()
results.verdad.isna().value_counts()
roc_auc_score(y_true=test.sepostulo, y_score=results.prediccion.values)

0.8230637264600005

In [11]:
joined.dtype

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [ ]:
avisos_detalle.nombre_zona.value_counts()

# Generar un resultado

In [ ]:
tentativa = pd.read_csv('datos/template_resultado.csv')
submit = pd.DataFrame()
submit['id'] = tentativa['id']
submit = submit.set_index('id')
submit['sepostulo'] =0
submit.to_csv(path_or_buf='submit.csv')
submit

# Defino los shingles de cada anuncio

In [ ]:
def shingles(string, n = 3):
    return [string[i:i + n] for i in range(len(string) - n + 1)]

def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    print(set(list1).intersection(list2))
    union = (len(list1) + len(list2)) - intersection
    print(union)
    return float(intersection / union)
#avisos_detalle['shingles_descripcion'] = avisos_detalle.descripcion.apply(shingles)
#avisos_detalle['shingles_titulo'] = avisos_detalle.titulo.apply(shingles)